## **Problem Statement**

To maintain transparency with their clients, food delivery services such as Zomato and Swiggy need to indicate the precise time it will take to deliver your order. Based on how long delivery partners have previously taken to deliver the same distance, these businesses employ machine learning algorithms to forecast when food will be delivered.

We must determine the distance between the location of food preparation and the location of food consumption in order to forecast the meal delivery time in real-time. Once the distance between the restaurant and the delivery locations has been determined, we must determine the correlations between the time it took delivery partners to deliver the food over the same distance in the past.


## **Understanding the data**

#### **Importing the required Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for ignoring warning
import warnings
warnings.filterwarnings("ignore")


# displaying each column and row
pd.set_option('display.max_columns',None)
pd.set_option("display.max_rows", None)

##### **Checking the data**

#### **Loading the dataset**

In [2]:
df = pd.read_csv("data/finalTrain.csv")

In [3]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [4]:
# checking the shape of the data
df.shape

(45584, 20)

- Number of rows: 45584
- Number of columns: 20

##### **Type and name of features in the data**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45584 non-null  object 
 1   Delivery_person_ID           45584 non-null  object 
 2   Delivery_person_Age          43730 non-null  float64
 3   Delivery_person_Ratings      43676 non-null  float64
 4   Restaurant_latitude          45584 non-null  float64
 5   Restaurant_longitude         45584 non-null  float64
 6   Delivery_location_latitude   45584 non-null  float64
 7   Delivery_location_longitude  45584 non-null  float64
 8   Order_Date                   45584 non-null  object 
 9   Time_Orderd                  43853 non-null  object 
 10  Time_Order_picked            45584 non-null  object 
 11  Weather_conditions           44968 non-null  object 
 12  Road_traffic_density         44983 non-null  object 
 13  Vehicle_conditio

In [6]:
categorical_count = 0
numerical_count = 0

for col in df.columns:
    if col in df.select_dtypes(include="object"):
        categorical_count += 1
    else:
        numerical_count += 1

print("Number of numerical features: ", numerical_count)
print("Number of categorical features: ", categorical_count)

Number of numerical features:  9
Number of categorical features:  11


- There are 11 categorical features
- 9 numerical features
    - 2 -> Interger value features
    - 7 -> Float value features

##### **Missing values**

In [7]:
df.isnull().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [8]:
# checking percentages of missing values in each feature
for col in df.columns:
    if df[col].isnull().sum() > 0:
        percentage = (df[col].isnull().sum()/df.shape[0])*100
        print(f"---------[{col}]---------")
        print(f"Number of missing values : {df[col].isnull().sum()}")
        print(f"Percentage : {round(percentage,2)}%\n")

---------[Delivery_person_Age]---------
Number of missing values : 1854
Percentage : 4.07%

---------[Delivery_person_Ratings]---------
Number of missing values : 1908
Percentage : 4.19%

---------[Time_Orderd]---------
Number of missing values : 1731
Percentage : 3.8%

---------[Weather_conditions]---------
Number of missing values : 616
Percentage : 1.35%

---------[Road_traffic_density]---------
Number of missing values : 601
Percentage : 1.32%

---------[multiple_deliveries]---------
Number of missing values : 993
Percentage : 2.18%

---------[Festival]---------
Number of missing values : 228
Percentage : 0.5%

---------[City]---------
Number of missing values : 1200
Percentage : 2.63%



As the total number of values are 45584, the missing values are quite high. 

##### **Checking for duplicated values**

In [9]:
df.duplicated().sum()

0

There no duplicates in the data.

##### **Checking unique values**

In [10]:
df.nunique()

ID                             45584
Delivery_person_ID              1320
Delivery_person_Age               22
Delivery_person_Ratings           28
Restaurant_latitude              657
Restaurant_longitude             518
Delivery_location_latitude      4373
Delivery_location_longitude     4373
Order_Date                        44
Time_Orderd                      176
Time_Order_picked                193
Weather_conditions                 6
Road_traffic_density               4
Vehicle_condition                  4
Type_of_order                      4
Type_of_vehicle                    4
multiple_deliveries                4
Festival                           2
City                               3
Time_taken (min)                  45
dtype: int64

In [11]:
# creating a dataframe for dataset understanding

def basicInfo_data(dataframe):
    ''' 
    This function will provide the basic information (number of missing values, missing value percentage, unique value count, datatype of the feature) about the dataset.
    Input: DataFrame
    Output: DataFrame
    '''
    Features = []
    Missing_values = []
    Unique_value_count = []
    Missing_values_percentage = []
    datatype = []

    for col in dataframe.columns:
        Features.append(col)
        Missing_values.append(dataframe[col].isnull().sum())
        Missing_values_percentage.append(round((dataframe[col].isnull().sum()/dataframe.shape[0])*100,2))
        Unique_value_count.append(dataframe[col].nunique())
        datatype.append(dataframe[col].dtype)

    # filling the dataframe
    temp_df = pd.DataFrame({
        "Features": Features,
        "Missing_values":Missing_values,
        "Missing_value_percentage":Missing_values_percentage,
        "Unique_value_count":Unique_value_count,
        "Datatype":datatype 
    }) 

    # display the dataframe
    return temp_df

In [12]:
basicInfo_data(dataframe=df)

,Features,Missing_values,Missing_value_percentage,Unique_value_count,Datatype
0,ID,0,0.00,45584,object
1,Delivery_person_ID,0,0.00,1320,object
2,Delivery_person_Age,1854,4.07,22,float64
3,Delivery_person_Ratings,1908,4.19,28,float64
4,Restaurant_latitude,0,0.00,657,float64
5,Restaurant_longitude,0,0.00,518,float64
6,Delivery_location_latitude,0,0.00,4373,float64
7,Delivery_location_longitude,0,0.00,4373,float64
8,Order_Date,0,0.00,44,object
9,Time_Orderd,1731,3.80,176,object


##### **Checking the statistics of the features**

In [13]:
df.describe()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_taken (min)
count,43730.000000,43676.000000,45584.000000,45584.000000,45584.000000,45584.000000,45584.000000,44591.000000,45584.000000
mean,29.566911,4.633774,17.017948,70.229684,17.465480,70.844161,1.023385,0.744635,26.293963
std,5.815064,0.334744,8.185674,22.885575,7.335562,21.120578,0.839055,0.572510,9.384298
min,15.000000,1.000000,-30.905562,-88.366217,0.010000,0.010000,0.000000,0.000000,10.000000
25%,25.000000,4.500000,12.933284,73.170000,12.988453,73.280000,0.000000,0.000000,19.000000
50%,30.000000,4.700000,18.551440,75.897963,18.633934,76.002574,1.000000,1.000000,26.000000
75%,35.000000,4.900000,22.728163,78.044095,22.785049,78.107044,2.000000,1.000000,32.000000
max,50.000000,6.000000,30.914057,88.433452,31.054057,88.563452,3.000000,3.000000,54.000000


**`Delivery_person_Age:`**

- With the presence of approx 4.07% of missing values the average age of the delivery person is 29.
- The maximum age is 50 years.
- The minimum age is 15 years.

**`Time_taken(min)`**

- The average time taken by the delivery person for the delivery is approx. 26 minutes.
- The maximum time taken is 54 minutes.
- Whereas the minimum time taken is 10 minutes, this might be becuse of the short distance between the restaurant location and the receiver's location.

## **Pre-processing**

In [14]:
# creating a copy of the data
data = df.copy()

In [15]:
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


Droping features that are not required in predicting the delivery time.

**Columns ->** ID, Delivery_person_Age

In [16]:
data.drop(columns=["ID","Delivery_person_Age"], axis=1 ,inplace=True)

In [17]:
data.head()

,Delivery_person_ID,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,DEHRES17DEL01,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,KOCRES16DEL01,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,PUNERES13DEL03,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,LUDHRES15DEL02,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,KNPRES14DEL02,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [18]:
data.shape

(45584, 18)

**Preprocessing the `Order_date` feature**

In [19]:
data['Order_Date'].value_counts()

Order_Date
15-03-2022    1192
03-04-2022    1178
13-03-2022    1169
26-03-2022    1165
24-03-2022    1162
09-03-2022    1159
05-04-2022    1156
05-03-2022    1154
07-03-2022    1153
03-03-2022    1150
21-03-2022    1149
11-03-2022    1148
19-03-2022    1148
30-03-2022    1141
01-03-2022    1140
28-03-2022    1139
17-03-2022    1134
01-04-2022    1133
02-03-2022    1012
10-03-2022     996
16-03-2022     995
20-03-2022     993
02-04-2022     992
06-03-2022     986
04-03-2022     980
29-03-2022     977
25-03-2022     974
14-03-2022     973
11-02-2022     970
18-03-2022     968
31-03-2022     967
27-03-2022     965
08-03-2022     964
12-03-2022     964
23-03-2022     964
06-04-2022     961
13-02-2022     957
15-02-2022     945
04-04-2022     941
17-02-2022     939
12-02-2022     864
16-02-2022     861
18-02-2022     855
14-02-2022     851
Name: count, dtype: int64

In [20]:
data['Order_Date'].dtype

dtype('O')

Since the dates stored in `Order_date` column are of object type, thus changing the datatype of this column. 

Creating 3 new columns from this column: 
- year
- month
- day

In [21]:
# Converting dtype
data['Order_Date'] = pd.to_datetime(data['Order_Date'], format="%d-%m-%Y")

# Creating new columns
data['Year'] = data['Order_Date'].dt.year
data['Month'] = data['Order_Date'].dt.month
data['Day'] = data['Order_Date'].dt.day

# droping the Order_date column
data.drop(columns='Order_Date', axis=1, inplace=True)

In [ ]:
data.head()

,Delivery_person_ID,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),Year,Month,Day
0,DEHRES17DEL01,4.2,30.327968,78.046106,30.397968,78.116106,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,2022,2,12
1,KOCRES16DEL01,4.7,10.003064,76.307589,10.043064,76.347589,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,2022,2,13
2,PUNERES13DEL03,4.7,18.562450,73.916619,18.652450,74.006619,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,2022,3,4
3,LUDHRES15DEL02,4.3,30.899584,75.809346,30.919584,75.829346,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,2022,2,13
4,KNPRES14DEL02,4.7,26.463504,80.372929,26.593504,80.502929,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,2022,2,14


In [23]:
basicInfo_data(dataframe=data)

,Features,Missing_values,Missing_value_percentage,Unique_value_count,Datatype
0,Delivery_person_ID,0,0.00,1320,object
1,Delivery_person_Ratings,1908,4.19,28,float64
2,Restaurant_latitude,0,0.00,657,float64
3,Restaurant_longitude,0,0.00,518,float64
4,Delivery_location_latitude,0,0.00,4373,float64
5,Delivery_location_longitude,0,0.00,4373,float64
6,Time_Orderd,1731,3.80,176,object
7,Time_Order_picked,0,0.00,193,object
8,Weather_conditions,616,1.35,6,object
9,Road_traffic_density,601,1.32,4,object


**Pre-processing `Time_Ordered`**

This column have 3.80% of missing values. Since I don't have any technique to fill those missing values; T am droping the missing values rows

In [24]:
data.dropna(subset=['Time_Orderd'], inplace=True)

In [25]:
data['Time_Orderd'].isnull().sum()

0

In [26]:
data['Time_Orderd'].value_counts()

Time_Orderd
21:55          461
17:55          456
0.833333333    449
22:20          448
21:35          446
19:50          444
21:15          442
22:45          438
21:20          437
18:35          436
19:55          436
18:10          435
17:35          435
21:25          435
22:10          434
23:30          434
17:40          431
17:25          431
1              430
21:40          429
23:25          429
22:35          428
22:50          425
18:20          425
0.791666667    424
17:15          424
21:10          423
20:40          423
20:45          423
0.958333333    422
23:35          422
18:30          421
17:45          421
20:50          420
19:25          420
19:30          419
23:40          419
20:35          416
20:20          416
19:40          416
23:10          414
17:20          414
21:45          413
19:15          413
19:35          412
20:30          412
17:10          412
18:40          411
23:55          411
17:50          410
22:30          410
0.875          409


There are some values within this column that are not written correctly. For example: - **0.958333333, 0.5** or **1** etc.<br> Thus correcting them.

In [27]:
def extract_time(x):
    try:
        return x.split(":")[0]+ ":"+ x.split(":")[1][:2]
    except IndexError:
        return "00:00"
    

In [28]:
data["Time_Orderd"] = data['Time_Orderd'].apply(extract_time)

In [29]:
data["Time_Orderd"].value_counts().head(10)

Time_Orderd
00:00    4068
21:55     461
17:55     456
22:20     448
21:35     446
19:50     444
21:15     442
22:45     438
21:20     437
19:55     436
Name: count, dtype: int64

Changing the datatype and format of Time_Orderd column

In [30]:
data["Time_Orderd"] = pd.to_datetime(data["Time_Orderd"], format="%H:%M", errors="coerce").dt.time

In [31]:
basicInfo_data(data)

,Features,Missing_values,Missing_value_percentage,Unique_value_count,Datatype
0,Delivery_person_ID,0,0.00,1320,object
1,Delivery_person_Ratings,268,0.61,26,float64
2,Restaurant_latitude,0,0.00,491,float64
3,Restaurant_longitude,0,0.00,389,float64
4,Delivery_location_latitude,0,0.00,4366,float64
5,Delivery_location_longitude,0,0.00,4366,float64
6,Time_Orderd,0,0.00,161,object
7,Time_Order_picked,0,0.00,193,object
8,Weather_conditions,0,0.00,6,object
9,Road_traffic_density,0,0.00,4,object


**Pre-processing `Time_Order_picked` column**

In [32]:
data["Time_Order_picked"].value_counts()

Time_Order_picked
21:30          483
22:50          453
21:45          445
17:55          442
18:40          441
22:25          441
23:50          438
18:05          438
22:40          437
23:35          434
20:50          434
21:55          431
0.833333333    431
18:45          430
23:05          428
18:25          428
19:55          427
19:40          426
19:35          425
17:35          423
17:45          421
20:40          421
20:55          420
1              420
20:30          418
24:05:00       417
21:05          417
19:50          417
17:25          416
23:45          416
21:50          416
20:25          415
0.916666667    415
18:50          411
17:50          410
20:05          410
17:30          410
19:30          409
22:30          409
19:25          409
21:40          409
22:45          409
0.75           408
17:40          407
21:35          406
22:55          406
23:25          405
22:05          404
18:30          403
18:55          401
18:35          401
23:30        

In [33]:
df['Time_Order_picked'] = df['Time_Order_picked'].str.replace('.', ':')

# Define a lambda function with error handling
def extract_time(x):
    try:
        return x.split(':')[0] + ':' + x.split(':')[1][:2]
    except IndexError:
        return '00:00'

# Apply the lambda function to the Order_Time column
df['Time_Order_picked'] = df['Time_Order_picked'].apply(extract_time)

# Convert the Order_Time column to a time data type
df['Time_Order_picked'] = pd.to_datetime(df['Time_Order_picked'], format='%H:%M:%S',errors='coerce').dt.time

In [34]:
basicInfo_data(data)

,Features,Missing_values,Missing_value_percentage,Unique_value_count,Datatype
0,Delivery_person_ID,0,0.00,1320,object
1,Delivery_person_Ratings,268,0.61,26,float64
2,Restaurant_latitude,0,0.00,491,float64
3,Restaurant_longitude,0,0.00,389,float64
4,Delivery_location_latitude,0,0.00,4366,float64
5,Delivery_location_longitude,0,0.00,4366,float64
6,Time_Orderd,0,0.00,161,object
7,Time_Order_picked,0,0.00,193,object
8,Weather_conditions,0,0.00,6,object
9,Road_traffic_density,0,0.00,4,object


**Pre-processing `multiple_deliveries` column**

In [35]:
data["multiple_deliveries"].value_counts()

multiple_deliveries
1.0    27060
0.0    13587
2.0     1920
3.0      343
Name: count, dtype: int64

The `multiple_deliveries` column is having 4 types of values (1.0, 0.0, 2.0 and 3.0). It has approx. 2.15% of missing values thus filling them with the mode.

In [36]:
data["multiple_deliveries"].fillna(data["multiple_deliveries"].mode()[0], inplace=True)

In [37]:
basicInfo_data(data)

,Features,Missing_values,Missing_value_percentage,Unique_value_count,Datatype
0,Delivery_person_ID,0,0.00,1320,object
1,Delivery_person_Ratings,268,0.61,26,float64
2,Restaurant_latitude,0,0.00,491,float64
3,Restaurant_longitude,0,0.00,389,float64
4,Delivery_location_latitude,0,0.00,4366,float64
5,Delivery_location_longitude,0,0.00,4366,float64
6,Time_Orderd,0,0.00,161,object
7,Time_Order_picked,0,0.00,193,object
8,Weather_conditions,0,0.00,6,object
9,Road_traffic_density,0,0.00,4,object


**Pre-processing `Festival` column**

In [38]:
data['Festival'].value_counts()

Festival
No     42777
Yes      857
Name: count, dtype: int64

Since, there is only 0.5% of missing values in `festival` column; I am dropping the rows having missing values.